In [ ]:
!pip install lightning

In [2]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

import lightning as pl
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch import Trainer
from torchmetrics import Accuracy

In [11]:
#NEURAL NET CLASS:::::
class ligNeuralNet(pl.LightningModule):
    def __init__(self):
        super(ligNeuralNet, self).__init__()
        self.input_size = 4
        self.l1 = nn.Linear(4, 12)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(12, 2)
        self.accuracy = Accuracy(task='multiclass', num_classes = 2)

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)

        return out
    #+===============================================================================================================+
    #TRAINING FUNCTIONS:
    def training_step(self, batch, batch_idx):
        features, target = batch
        #print(type(batch))
        # Forward pass
        outputs = self(features)
        loss = F.cross_entropy(outputs, target)

        self.accuracy(outputs, target)
        self.log('acc', self.accuracy, prog_bar=True)
        self.log('loss', loss)
        return loss

    # define what happens for testing here

    #def train_dataloader(self):
        # Data loader
    #    return torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, num_workers=4, shuffle=True)
    #+===============================================================================================================+
    #VALIDATION FUNCTIONS:
    #def val_dataloader(self):
    #    return torch.utils.data.DataLoader(dataset = val_dataset, batch_size=batch_size, num_workers=4, shuffle=False)

    def validation_step(self, batch, batch_idx):
        inputs, target = batch
        outputs = self.forward(inputs)
        self.accuracy(outputs, target)
        self.log('val_acc', self.accuracy, prog_bar=True)

    #+===============================================================================================================+

    #+===============================================================================================================+
    #test functions:
    #def predict_dataloader(self):##############
    #    return torch.utils.data.DataLoader(dataset = test_dataset, batch_size=batch_size, num_workers=4, shuffle=False)

    def predict_step(self, batch, batch_idx):
        features, target = batch
        y_pred= self(features)
        #for x in y_pred:
        #    print(torch.argmax(x))
        return y_pred.softmax(dim=-1)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

In [76]:
class PredictionDataSet(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        target = torch.tensor([0])
        features = torch.tensor(self.data.iloc[idx, :4], dtype=torch.float32)
        return features, target

In [83]:
class BlackjackPredictor:
  """
  A class for predicting Blackjack game outcomes using a trained neural network model.

  Note:
  Blackjack is generally random! It is difficult to predict the best choice when there is little consistancy in results
  As well, the default model (blackjack_model.ckpt) was trained only making use of 4 inputs. (Card A, Card B, DidHit, and Dealer Card)
  which results in inaccuracys since the network does not base its answers off cards gained after hitting.

  Parameters:
  - checkpoint (str): Path to the checkpoint file for the pre-trained model. Default is "./blackjack_model.ckpt".
  """

  def __init__(self, checkpoint="./blackjack_model.ckpt"):
    """
    Initializes the BlackjackPredictor object with a pre-trained model.

    Parameters:
    - checkpoint (str): Path to the checkpoint file for the pre-trained model. Default is "./blackjack_model.ckpt".
    """
    self.model = ligNeuralNet.load_from_checkpoint(checkpoint)
    self.model.eval()


  def predict(self, data: pd.DataFrame):
    """
    Predicts the outcome of Blackjack games based on the input data.

    Parameters:
    - data (pd.DataFrame): Input data containing information about cards and game state.

    Returns:
    - torch.Tensor: Predicted outcomes for each input sample.
    """
    trainer = Trainer()
    dataloader = torch.utils.data.DataLoader(dataset=PredictionDataSet(data), batch_size=1, num_workers=1, shuffle=False)
    result = trainer.predict(self.model, dataloader, return_predictions=True)
    return result


  def getWinChance(self, card1: int, card2: int, dealerCard: int, doesHit: bool):
    """
    Calculates the winning chance based on the given cards and game state.

    Parameters:
    - card1 (int): First player card.
    - card2 (int): Second player card.
    - dealerCard (int): Dealer's face-up card.
    - doesHit (bool): Indicates whether the player chooses to hit or stand.

    Returns:
    - torch.Tensor: Winning chance for the specified scenario.
    """
    frame = pd.DataFrame([[card1, card2, doesHit, dealerCard]])
    return self.predict(frame)


  def shouldHit(self, card1: int, card2: int, dealerCard: int):
    """
    Determines whether the player should hit or stand based on the winning chances.

    Parameters:
    - card1 (int): First player card.
    - card2 (int): Second player card.
    - dealerCard (int): Dealer's face-up card.

    Returns:
    - bool: True if the player should hit, False if the player should stand.
    """
    hitResult = self.getWinChance(card1, card2, dealerCard, True)
    standResult = self.getWinChance(card1, card2, dealerCard, False)
    return (hitResult[0][0][0] > standResult[0][0][0]).item()

In [108]:
predictor = BlackjackPredictor()

predictor.shouldHit(4, 4, 7)

INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

False